In [1]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

In [2]:
import os, sys, re, collections, string, itertools
from operator import itemgetter as at
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook
sys.path.append("../python")
import data
%matplotlib inline
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from gensim.models import Phrases
import nltk
from nltk.stem.porter import PorterStemmer
# SKLearn
from sklearn.feature_extraction import text
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from ipywidgets import interact

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ishay\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ishay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import os, re
import pandas as pd
from tqdm import tqdm
import pickle

def listTestFiles():
    """Returns a list of all test data in the data/ folder"""
    return [f for f in os.listdir("../data") if f.endswith(".txt") and f.split(".")[0].isdigit()]


def listFiles():
    """Returns a list of all training data in the data/ folder"""
    return [f for f in os.listdir("../data") if f.endswith(".txt") and f.find("-")>0]

def readFile(fname):
    """Given a file name `fname` or a list of filenames, returns its content"""
    if type(fname) == str:
        try:
            with open("../data/"+fname, 'rb') as f:
                data  = f.read().decode("utf8", "ignore")
        except:
            data = None
        return data
    elif type(fname) == list:
        return list(map(readFile, fname))
    else:
        raise TypeError("fname should be a string or a list of strings")

def getIterator():
    """Iterate over all training files amd get their content"""
    for fname in listFiles():
        tag = fname.split("-", 1)[0]
        yield tag, readFile(fname)

def asDataFrame(vectorizer=None, full_text=None, vocabulary=None):
    """
    Return a pandas DataFrame of all files,
    If a vectorizer function is passed, apply it on the textual data
    """
    ret = []
    if vectorizer is None:
        cols = ["num", "file", "tag"]
    else:
        cols = ["num", "vector", "tag"]
    for fname in tqdm(listFiles()):
        if fname.find("-")<0:
            continue
        tag, ind = fname.split("-", 1)
        if vectorizer is None:
            ret.append((int(ind.split(".", 1)[0]), fname, tag))
        else:
            ret.append((int(ind.split(".", 1)[0]), vectorizer(readFile(fname)), tag)) # full_text, vocabulary
    return pd.DataFrame(ret, columns=cols).set_index("num")

def preprocessed():
    """
    Return the training data and label, after replacing numbers with the <NUM> token, and removing non alpha numeric characters
    """
    X = []
    y = []
    digits = re.compile(r"\d[\d\.\$]*")
    not_allowed = re.compile(r"[^\s\w<>]")
    clean = lambda text: not_allowed.sub("", digits.sub("<NUM>",text.lower()))
    for fname in listFiles():
        tag, ind = fname.split("-", 1)
        body = clean(readFile(fname))
        y.append(tag)
        X.append(body)
    return (X,y)


def stemmed():
    """Apply Porter stemming on all documents in the training data, and return training data and label"""
    if os.path.exists("../data/stemmed_x.pickle"):
        with open("../data/stemmed_x.pickle", "rb") as f:
            X = pickle.load(f)
        with open("../data/stemmed_y.pickle", "rb") as f:
            y = pickle.load(f)
        return (X,y)
    import nltk
    from nltk.stem.porter import PorterStemmer
    porter = PorterStemmer()
    X = []
    y = []
    punc = re.compile(r"[\.,;\(\)\s]+")
    not_allowed = re.compile(r"[^\sa-z]")
    clean = lambda text: not_allowed.sub("", punc.sub(" ",text.lower()))
    for fname in tqdm(listFiles()):
        if fname.find("-")<0:
            continue
        tag, ind = fname.split("-", 1)
        body = clean(readFile(fname))
        body = " ".join([porter.stem(w) for w in body.split()])
        y.append(tag)
        X.append(body)
    with open("../data/stemmed_x.pickle", "wb") as f:
        pickle.dump(X, f)
    with open("../data/stemmed_y.pickle", "wb") as f:
        pickle.dump(y, f)
    return (X,y)


In [4]:
df = asDataFrame()
df.head()

100%|██████████████████████████████████████████████████████████████████████████| 4427/4427 [00:00<00:00, 459631.26it/s]


,file,tag
num,,
39500,bylaws-39500.txt,bylaws
39510,bylaws-39510.txt,bylaws
39520,bylaws-39520.txt,bylaws
39530,bylaws-39530.txt,bylaws
39540,bylaws-39540.txt,bylaws


In [5]:
df.shape

(4427, 2)

In [6]:
# Get all the text (from all of the files) for the TFidf vectorizer (for the idf part)
full_text = ""
for file_name in tqdm(df["file"][:100]):
    file_text = readFile(file_name)
    full_text += file_text 
full_text

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 668.30it/s]


'Exhibit 3.1\n\nNightingale, Inc.\n\nForm 8-K\n\nFile No. 33-23429-D\n\nAMENDED AND RESTATED\n\n\n\nOF\n\nNIGHTINGALE, INC.\n\n  \n\n\nNightingale, Inc. (the “Corporation”) was formed and is existing under the Utah Revised Business Corporation Act (“URBCA”). The Corporation hereby adopts these Amended and Restated Articles of Incorporation, which restate, integrate and amend the Corporation\'s Articles of Incorporation. These Amended and Restated Articles of Incorporation, including the amendments adopted hereby, have been duly adopted by the Board of Directors and Shareholders of the Corporation in accordance with the provisions of Section 16-10a-1003, Section 16-10a-1006 and Section 16-10a-1007 of the URBCA.\n\nADOPTION OF AMENDED AND RESTATED ARTICLES OF INCORPORATION\n\nThe Amended and Restated Articles of Incorporation set forth below were duly adopted by the written consent of shareholders of the Corporation in lieu of a meeting of shareholders held in the manner prescribed by th

In [7]:
type(full_text)

str

In [8]:
stemmer = PorterStemmer()
plain_cv = CountVectorizer()
analyzer = plain_cv.build_analyzer()
def my_analyzer(txt):
    return [stemmer.stem(w) for w in analyzer(txt)]

In [9]:
word_count = collections.Counter()

In [10]:
for index, row in tqdm_notebook(df.iloc[:100,:].iterrows()):
    fname = row["file"]
    with open("../data/"+fname, 'rb') as f:
        data  = f.read().decode("utf8", "ignore")
        # Now replacing with num tokens
        digits = re.compile(r"\d[\d\$\%\.]*")
        data_no_digits = digits.sub("<NUM>", data)
        # Now vectorizing
        cv_with_stem = CountVectorizer(stop_words='english', analyzer=my_analyzer, ngram_range=(1,1))
        cv_with_stem.fit([data_no_digits])
        trans_data = cv_with_stem.transform([data_no_digits])
        print(trans_data)
        #print(cv_with_stem.get_feature_names())
        #doc_count = {k:v for k,v in zip(cv_with_stem.get_feature_names(), trans_data.toarray()[0])}
        word_count += collections.Counter(cv_with_stem.get_feature_names())

A Jupyter Widget

  (0, 0)	1
  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	2
  (0, 5)	1
  (0, 6)	3
  (0, 7)	3
  (0, 8)	1
  (0, 9)	2
  (0, 10)	1
  (0, 11)	9
  (0, 12)	1
  (0, 13)	1
  (0, 14)	2
  (0, 15)	1
  (0, 16)	1
  (0, 17)	1
  (0, 18)	1
  (0, 19)	4
  (0, 20)	26
  (0, 21)	3
  (0, 22)	3
  (0, 23)	57
  (0, 24)	19
  :	:
  (0, 286)	6
  (0, 287)	3
  (0, 288)	5
  (0, 289)	1
  (0, 290)	1
  (0, 291)	2
  (0, 292)	2
  (0, 293)	2
  (0, 294)	2
  (0, 295)	1
  (0, 296)	21
  (0, 297)	2
  (0, 298)	1
  (0, 299)	1
  (0, 300)	5
  (0, 301)	10
  (0, 302)	1
  (0, 303)	2
  (0, 304)	2
  (0, 305)	1
  (0, 306)	1
  (0, 307)	8
  (0, 308)	1
  (0, 309)	2
  (0, 310)	2
  (0, 0)	1
  (0, 1)	7
  (0, 2)	1
  (0, 3)	1
  (0, 4)	4
  (0, 5)	1
  (0, 6)	4
  (0, 7)	8
  (0, 8)	9
  (0, 9)	2
  (0, 10)	1
  (0, 11)	2
  (0, 12)	6
  (0, 13)	2
  (0, 14)	1
  (0, 15)	6
  (0, 16)	1
  (0, 17)	1
  (0, 18)	2
  (0, 19)	1
  (0, 20)	3
  (0, 21)	4
  (0, 22)	17
  (0, 23)	23
  (0, 24)	2
  :	:
  (0, 534)	1
  (0, 535)	1
  (0, 536)	2
  (0, 537)	2
  (0, 538)	20
  (0, 539)

  (0, 474)	10
  (0, 0)	2
  (0, 1)	2
  (0, 2)	7
  (0, 3)	1
  (0, 4)	8
  (0, 5)	15
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	2
  (0, 10)	2
  (0, 11)	2
  (0, 12)	2
  (0, 13)	1
  (0, 14)	6
  (0, 15)	6
  (0, 16)	1
  (0, 17)	1
  (0, 18)	2
  (0, 19)	1
  (0, 20)	6
  (0, 21)	2
  (0, 22)	2
  (0, 23)	3
  (0, 24)	14
  :	:
  (0, 410)	3
  (0, 411)	1
  (0, 412)	4
  (0, 413)	5
  (0, 414)	3
  (0, 415)	2
  (0, 416)	3
  (0, 417)	1
  (0, 418)	1
  (0, 419)	1
  (0, 420)	1
  (0, 421)	12
  (0, 422)	1
  (0, 423)	6
  (0, 424)	2
  (0, 425)	1
  (0, 426)	1
  (0, 427)	1
  (0, 428)	11
  (0, 429)	1
  (0, 430)	5
  (0, 431)	3
  (0, 432)	6
  (0, 433)	3
  (0, 434)	6
  (0, 0)	1
  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	4
  (0, 7)	1
  (0, 8)	6
  (0, 9)	1
  (0, 10)	1
  (0, 11)	1
  (0, 12)	1
  (0, 13)	2
  (0, 14)	1
  (0, 15)	3
  (0, 16)	2
  (0, 17)	1
  (0, 18)	1
  (0, 19)	3
  (0, 20)	2
  (0, 21)	1
  (0, 22)	9
  (0, 23)	1
  (0, 24)	1
  :	:
  (0, 124)	2
  (0, 125)	24
  (0, 126)	1
  (0, 127)	8
  (0, 128)	6

  (0, 434)	2
  (0, 0)	1
  (0, 1)	1
  (0, 2)	6
  (0, 3)	2
  (0, 4)	1
  (0, 5)	4
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 10)	8
  (0, 11)	22
  (0, 12)	1
  (0, 13)	1
  (0, 14)	3
  (0, 15)	4
  (0, 16)	2
  (0, 17)	4
  (0, 18)	2
  (0, 19)	2
  (0, 20)	2
  (0, 21)	2
  (0, 22)	5
  (0, 23)	7
  (0, 24)	1
  :	:
  (0, 512)	1
  (0, 513)	2
  (0, 514)	42
  (0, 515)	1
  (0, 516)	1
  (0, 517)	4
  (0, 518)	2
  (0, 519)	2
  (0, 520)	5
  (0, 521)	3
  (0, 522)	4
  (0, 523)	18
  (0, 524)	1
  (0, 525)	10
  (0, 526)	2
  (0, 527)	4
  (0, 528)	2
  (0, 529)	11
  (0, 530)	5
  (0, 531)	17
  (0, 532)	1
  (0, 533)	1
  (0, 534)	8
  (0, 535)	9
  (0, 536)	3
  (0, 0)	1
  (0, 1)	3
  (0, 2)	3
  (0, 3)	6
  (0, 4)	2
  (0, 5)	1
  (0, 6)	7
  (0, 7)	6
  (0, 8)	2
  (0, 9)	3
  (0, 10)	7
  (0, 11)	10
  (0, 12)	8
  (0, 13)	2
  (0, 14)	2
  (0, 15)	1
  (0, 16)	5
  (0, 17)	6
  (0, 18)	3
  (0, 19)	1
  (0, 20)	1
  (0, 21)	27
  (0, 22)	2
  (0, 23)	1
  (0, 24)	2
  :	:
  (0, 496)	1
  (0, 497)	8
  (0, 498)	2
  (0, 499)	41
  (0, 500

  (0, 495)	3
  (0, 0)	2
  (0, 1)	4
  (0, 2)	1
  (0, 3)	1
  (0, 4)	5
  (0, 5)	1
  (0, 6)	4
  (0, 7)	2
  (0, 8)	8
  (0, 9)	12
  (0, 10)	2
  (0, 11)	1
  (0, 12)	2
  (0, 13)	5
  (0, 14)	1
  (0, 15)	4
  (0, 16)	3
  (0, 17)	1
  (0, 18)	2
  (0, 19)	7
  (0, 20)	3
  (0, 21)	2
  (0, 22)	2
  (0, 23)	1
  (0, 24)	6
  :	:
  (0, 467)	1
  (0, 468)	2
  (0, 469)	2
  (0, 470)	21
  (0, 471)	5
  (0, 472)	2
  (0, 473)	1
  (0, 474)	1
  (0, 475)	1
  (0, 476)	6
  (0, 477)	1
  (0, 478)	2
  (0, 479)	11
  (0, 480)	14
  (0, 481)	5
  (0, 482)	1
  (0, 483)	12
  (0, 484)	1
  (0, 485)	5
  (0, 486)	9
  (0, 487)	1
  (0, 488)	3
  (0, 489)	1
  (0, 490)	4
  (0, 491)	7
  (0, 0)	1
  (0, 1)	1
  (0, 2)	1
  (0, 3)	5
  (0, 4)	5
  (0, 5)	2
  (0, 6)	2
  (0, 7)	10
  (0, 8)	21
  (0, 9)	6
  (0, 10)	8
  (0, 11)	1
  (0, 12)	8
  (0, 13)	5
  (0, 14)	1
  (0, 15)	1
  (0, 16)	1
  (0, 17)	2
  (0, 18)	1
  (0, 19)	2
  (0, 20)	7
  (0, 21)	2
  (0, 22)	8
  (0, 23)	1
  (0, 24)	1
  :	:
  (0, 617)	1
  (0, 618)	2
  (0, 619)	3
  (0, 620)	1
  (0, 621)	

  (0, 496)	4
  (0, 0)	3
  (0, 1)	1
  (0, 2)	1
  (0, 3)	2
  (0, 4)	2
  (0, 5)	3
  (0, 6)	1
  (0, 7)	1
  (0, 8)	2
  (0, 9)	1
  (0, 10)	1
  (0, 11)	2
  (0, 12)	2
  (0, 13)	1
  (0, 14)	1
  (0, 15)	1
  (0, 16)	1
  (0, 17)	1
  (0, 18)	1
  (0, 19)	2
  (0, 20)	1
  (0, 21)	1
  (0, 22)	1
  (0, 23)	1
  (0, 24)	1
  :	:
  (0, 43)	1
  (0, 44)	1
  (0, 45)	4
  (0, 46)	1
  (0, 47)	1
  (0, 48)	1
  (0, 49)	1
  (0, 50)	1
  (0, 51)	1
  (0, 52)	2
  (0, 53)	1
  (0, 54)	1
  (0, 55)	2
  (0, 56)	3
  (0, 57)	5
  (0, 58)	4
  (0, 59)	1
  (0, 60)	2
  (0, 61)	10
  (0, 62)	2
  (0, 63)	1
  (0, 64)	1
  (0, 65)	5
  (0, 66)	2
  (0, 67)	1
  (0, 0)	2
  (0, 1)	3
  (0, 2)	4
  (0, 3)	2
  (0, 4)	2
  (0, 5)	1
  (0, 6)	13
  (0, 7)	14
  (0, 8)	1
  (0, 9)	2
  (0, 10)	8
  (0, 11)	7
  (0, 12)	1
  (0, 13)	3
  (0, 14)	2
  (0, 15)	1
  (0, 16)	2
  (0, 17)	1
  (0, 18)	1
  (0, 19)	4
  (0, 20)	4
  (0, 21)	2
  (0, 22)	2
  (0, 23)	1
  (0, 24)	19
  :	:
  (0, 526)	2
  (0, 527)	1
  (0, 528)	2
  (0, 529)	1
  (0, 530)	1
  (0, 531)	1
  (0, 532)	1


  (0, 565)	1
  (0, 0)	2
  (0, 1)	2
  (0, 2)	1
  (0, 3)	2
  (0, 4)	5
  (0, 5)	10
  (0, 6)	1
  (0, 7)	2
  (0, 8)	1
  (0, 9)	1
  (0, 10)	6
  (0, 11)	4
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (0, 15)	5
  (0, 16)	2
  (0, 17)	1
  (0, 18)	1
  (0, 19)	1
  (0, 20)	1
  (0, 21)	4
  (0, 22)	1
  (0, 23)	7
  (0, 24)	1
  :	:
  (0, 112)	7
  (0, 113)	31
  (0, 114)	1
  (0, 115)	3
  (0, 116)	2
  (0, 117)	1
  (0, 118)	1
  (0, 119)	2
  (0, 120)	14
  (0, 121)	1
  (0, 122)	1
  (0, 123)	2
  (0, 124)	2
  (0, 125)	1
  (0, 126)	2
  (0, 127)	1
  (0, 128)	1
  (0, 129)	2
  (0, 130)	1
  (0, 131)	1
  (0, 132)	1
  (0, 133)	1
  (0, 134)	1
  (0, 135)	1
  (0, 136)	1
  (0, 0)	1
  (0, 1)	1
  (0, 2)	2
  (0, 3)	6
  (0, 4)	1
  (0, 5)	4
  (0, 6)	1
  (0, 7)	1
  (0, 8)	3
  (0, 9)	1
  (0, 10)	2
  (0, 11)	1
  (0, 12)	4
  (0, 13)	2
  (0, 14)	2
  (0, 15)	4
  (0, 16)	13
  (0, 17)	1
  (0, 18)	3
  (0, 19)	6
  (0, 20)	47
  (0, 21)	25
  (0, 22)	1
  (0, 23)	1
  (0, 24)	1
  :	:
  (0, 310)	2
  (0, 311)	4
  (0, 312)	1
  (0, 313)	1
  (0, 314)	2

  (0, 508)	3
  (0, 0)	3
  (0, 1)	1
  (0, 2)	1
  (0, 3)	6
  (0, 4)	1
  (0, 5)	1
  (0, 6)	5
  (0, 7)	13
  (0, 8)	1
  (0, 9)	1
  (0, 10)	2
  (0, 11)	7
  (0, 12)	10
  (0, 13)	1
  (0, 14)	6
  (0, 15)	1
  (0, 16)	2
  (0, 17)	6
  (0, 18)	2
  (0, 19)	1
  (0, 20)	4
  (0, 21)	4
  (0, 22)	5
  (0, 23)	9
  (0, 24)	1
  :	:
  (0, 571)	25
  (0, 572)	2
  (0, 573)	7
  (0, 574)	3
  (0, 575)	1
  (0, 576)	2
  (0, 577)	2
  (0, 578)	27
  (0, 579)	3
  (0, 580)	8
  (0, 581)	3
  (0, 582)	5
  (0, 583)	9
  (0, 584)	1
  (0, 585)	1
  (0, 586)	1
  (0, 587)	1
  (0, 588)	1
  (0, 589)	1
  (0, 590)	1
  (0, 591)	1
  (0, 592)	1
  (0, 593)	9
  (0, 594)	1
  (0, 595)	11
  (0, 0)	1
  (0, 1)	2
  (0, 2)	1
  (0, 3)	4
  (0, 4)	1
  (0, 5)	9
  (0, 6)	2
  (0, 7)	1
  (0, 8)	1
  (0, 9)	2
  (0, 10)	4
  (0, 11)	2
  (0, 12)	5
  (0, 13)	4
  (0, 14)	2
  (0, 15)	3
  (0, 16)	3
  (0, 17)	2
  (0, 18)	1
  (0, 19)	8
  (0, 20)	1
  (0, 21)	4
  (0, 22)	3
  (0, 23)	1
  (0, 24)	18
  :	:
  (0, 510)	7
  (0, 511)	3
  (0, 512)	39
  (0, 513)	2
  (0, 514)	

  (0, 487)	8
  (0, 0)	1
  (0, 1)	7
  (0, 2)	3
  (0, 3)	1
  (0, 4)	2
  (0, 5)	2
  (0, 6)	9
  (0, 7)	21
  (0, 8)	3
  (0, 9)	7
  (0, 10)	11
  (0, 11)	4
  (0, 12)	5
  (0, 13)	2
  (0, 14)	1
  (0, 15)	3
  (0, 16)	2
  (0, 17)	4
  (0, 18)	6
  (0, 19)	2
  (0, 20)	23
  (0, 21)	1
  (0, 22)	1
  (0, 23)	1
  (0, 24)	1
  :	:
  (0, 497)	8
  (0, 498)	1
  (0, 499)	1
  (0, 500)	13
  (0, 501)	1
  (0, 502)	1
  (0, 503)	47
  (0, 504)	2
  (0, 505)	4
  (0, 506)	1
  (0, 507)	1
  (0, 508)	6
  (0, 509)	1
  (0, 510)	3
  (0, 511)	4
  (0, 512)	21
  (0, 513)	8
  (0, 514)	5
  (0, 515)	12
  (0, 516)	4
  (0, 517)	21
  (0, 518)	1
  (0, 519)	7
  (0, 520)	11
  (0, 521)	6
  (0, 0)	1
  (0, 1)	1
  (0, 2)	2
  (0, 3)	2
  (0, 4)	9
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	3
  (0, 9)	4
  (0, 10)	1
  (0, 11)	1
  (0, 12)	1
  (0, 13)	1
  (0, 14)	6
  (0, 15)	1
  (0, 16)	3
  (0, 17)	1
  (0, 18)	1
  (0, 19)	1
  (0, 20)	13
  (0, 21)	1
  (0, 22)	15
  (0, 23)	3
  (0, 24)	48
  :	:
  (0, 345)	1
  (0, 346)	1
  (0, 347)	1
  (0, 348)	1
  (0, 

In [11]:
word_count

Counter({'_____': 2,
         '_________': 1,
         '________________': 2,
         '________________________________': 1,
         'accord': 68,
         'accru': 7,
         'act': 95,
         'action': 84,
         'activ': 30,
         'addit': 69,
         'adjust': 11,
         'adopt': 91,
         'affect': 49,
         'affirm': 62,
         'after': 83,
         'against': 61,
         'agent': 79,
         'all': 91,
         'allow': 36,
         'alter': 65,
         'amend': 96,
         'amount': 62,
         'an': 89,
         'and': 100,
         'ani': 95,
         'annual': 79,
         'appeal': 14,
         'applic': 72,
         'approv': 72,
         'are': 92,
         'aris': 33,
         'articl': 95,
         'as': 98,
         'asset': 39,
         'at': 92,
         'author': 96,
         'automat': 11,
         'ballot': 39,
         'be': 98,
         'been': 85,
         'belief': 5,
         'below': 27,
         'best': 34,
         'board': 97,
  

In [12]:
total_word_count = sum(word_count.values())
word_count_hist = collections.Counter(word_count.values())
print ("Total word count: "+str(total_word_count))

Total word count: 45020


In [13]:
word_count_hist.items()

dict_items([(2, 257), (1, 922), (68, 9), (7, 59), (95, 5), (84, 4), (30, 15), (69, 6), (11, 31), (91, 7), (49, 7), (62, 5), (83, 7), (61, 7), (79, 7), (36, 7), (65, 5), (96, 8), (89, 6), (100, 7), (14, 25), (72, 9), (92, 6), (33, 15), (98, 6), (39, 13), (85, 8), (5, 74), (27, 14), (34, 8), (97, 5), (51, 6), (78, 12), (59, 3), (75, 8), (48, 4), (77, 6), (21, 12), (4, 100), (53, 6), (45, 10), (67, 13), (76, 5), (81, 8), (19, 17), (10, 50), (15, 26), (90, 5), (82, 5), (60, 7), (9, 43), (93, 5), (88, 4), (35, 9), (26, 11), (43, 6), (73, 7), (16, 28), (25, 18), (37, 12), (66, 7), (28, 13), (86, 4), (87, 6), (57, 13), (23, 19), (70, 6), (80, 4), (17, 18), (38, 15), (22, 14), (94, 9), (41, 7), (56, 11), (3, 159), (6, 73), (99, 1), (55, 11), (42, 6), (29, 28), (46, 14), (44, 6), (20, 22), (13, 24), (64, 6), (71, 8), (18, 25), (8, 35), (47, 7), (74, 7), (50, 8), (58, 6), (54, 8), (40, 8), (12, 25), (63, 9), (32, 10), (31, 11), (24, 22), (52, 8)])

In [14]:
# word_count_list = list(word_count_hist.items())
# X = [tup[0] for tup in word_count_list]
# y = [tup[1] for tup in word_count_list]
# plt.plot(X,y, '.b')

In [15]:
@interact(lb = (0,30), ub =(60,250))
def vocab_coverage(lb=0,ub=250):
    words_covered = sum([wc*n for wc, n in word_count_hist.items() if lb<wc<ub])
    corpus_percentage = words_covered/total_word_count
    vector_size = len([1 for wc, n in word_count_hist.items() if lb<wc<ub])
    return "Corpus Coverage: {c:.2f}%\n Vector Size: {v}".format(c=corpus_percentage*100,v=vector_size)

A Jupyter Widget

In [16]:
len(word_count)

2703

In [17]:
word_count_dict = dict(word_count)

In [18]:
word_count_reduced = dict({k: v for k, v in word_count_dict.items() if v < 200 and v >= 10})

In [19]:
vocab = list(word_count_reduced.keys())

In [20]:
len(vocab)

981

In [21]:
vocab[:10]

['accord',
 'act',
 'action',
 'activ',
 'addit',
 'adjust',
 'adopt',
 'affect',
 'affirm',
 'after']

### Bag of Words Vector

In [22]:
def bag_of_words(text):
    return collections.Counter(text.split())

In [23]:
vocabulary = vocab

In [24]:
# If I use this function for vectorizing, I need to turn it into a series after the genertor (in later stage)
def BOW_vector(text):
    yield at(*vocabulary)(bag_of_words(text))

### CountVectorizer Vector

In [25]:
def cv_vector(text, vocabulary):
    cv = CountVectorizer(stop_words='english', analyzer=my_analyzer, ngram_range=(1,1))
    cv.fit([text])
    text_trans = cv.transform([text])
    cv_dict = {k:v for k,v in zip(cv.get_feature_names(), text_trans.toarray()[0]) if v>0}
    return tuple([cv_dict[item] if item in cv_dict.keys() else 0 for item in vocabulary])

### TfidfVectrizer

In [26]:
tf = TfidfVectorizer(stop_words='english', analyzer=my_analyzer)
tf.fit([full_text]) # needs to fit on all of the documents

TfidfVectorizer(analyzer=<function my_analyzer at 0x00000208F4B676A8>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), norm='l2',
        preprocessor=None, smooth_idf=True, stop_words='english',
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [27]:
def tf_vector(text, tf, vocabulary):
    text_trans = tf.transform([text])
    tf_dict = {k:v for k,v in zip(tf.get_feature_names(), text_trans.toarray()[0]) if v>0.0001}
    return tuple([tf_dict[item] if item in tf_dict.keys() else 0 for item in vocabulary])

In [29]:
# Now I have several options as vectorizer: BOW_vector, cv_vector, tf_vector 

# df_vect = asDataFrame(vectorizer=tf_vector, full_text=tf, vocabulary=vocab)
# df_vect

In [31]:
# y = df_vect.tag

In [33]:
# X = df_vect.vector.apply(pd.Series) 

In [34]:
# X[:5]

In [35]:
# # This is required only when I use BOW_vector, when it returns a generator
# X = X.loc[:,0].apply(lambda x: pd.Series(x))

In [36]:
# # This is required only when I use BOW_vector, when it returns a generator
# X[:5]

In [39]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=0)

In [38]:
# lr_clf = LogisticRegression()

In [ ]:
# lr_clf.fit(X_train,y_train)

In [ ]:
# pred = lr_clf.predict(X_test)

In [ ]:
# acc = accuracy_score(y_test, pred)
# acc

In [ ]:
# cm = confusion_matrix(y_test, pred)
# cm

In [ ]:
# nb_clf = MultinomialNB()

# nb_clf.fit(X_train,y_train)

# pred = nb_clf.predict(X_test)
# acc = accuracy_score(y_test, pred)
# acc

### w2v

In [40]:
# Examine the first document in the df dataframe
text_first = df.iloc[0,0]

In [41]:
w2v = Word2Vec.load("../data/w2v.pickle")

In [42]:
# with open("../data/"+text_first, 'rb') as f:
#         data  = f.read().decode("utf8", "ignore")

In [43]:
# tokenized = nltk.word_tokenize(data)

In [44]:
# tokenized[:5]

In [45]:
from nltk.corpus import stopwords

In [46]:
# preprocessing text (manually), here only for the first document in the corpus
stemmer = PorterStemmer()
with open("../data/"+text_first, 'rb') as f:
        data  = f.read().decode("utf8", "ignore")
        # Now replacing with num tokens
        digits = re.compile(r"\d[\d\$\%\.]*")
        data_no_digits = digits.sub("<NUM>", data)
        # Take out not allowed characters
        not_allowed = re.compile(r"[^\s\w<>_]")
        data_allowed = not_allowed.sub("", data_no_digits)
        # Split to wirds
        words = data_no_digits.split()
        stemmed_words = [stemmer.stem(w) for w in words if w not in stopwords.words('english')] 

In [47]:
doc_vec = None
for word in stemmed_words:
    try:
        if doc_vec is not None:
            doc_vec += w2v.wv[word]
            n+=1
        else:
            doc_vec=np.copy(w2v.wv[word])
            n=1
    except KeyError:
        continue

doc_vec = doc_vec/n

In [48]:
doc_vec

array([ -1.61932695e+00,  -4.64450717e-01,   5.50537705e-02,
        -2.97372609e-01,  -8.05998981e-01,   8.71752620e-01,
        -7.41109192e-01,   6.78718030e-01,   3.10185850e-01,
         5.92026711e-01,   2.86219120e-01,   1.57633364e+00,
         5.07278800e-01,  -2.20609856e+00,  -1.87105447e-01,
        -1.86528161e-01,  -1.78492114e-01,   1.12959194e+00,
        -4.25157189e-01,   8.33562672e-01,  -1.58629620e+00,
        -5.41796733e-04,   9.10302818e-01,  -6.76452462e-03,
         1.02169240e+00,   4.13865000e-01,  -1.20437607e-01,
        -1.03570306e+00,   1.98027468e+00,  -3.50530855e-02,
        -4.81930822e-01,  -1.70742631e+00,   7.30840564e-01,
        -5.97360253e-01,   2.57509917e-01,   2.76606195e-02,
        -2.51242518e-01,  -8.97911072e-01,  -5.64375520e-01,
         1.20119345e+00,  -1.67658818e+00,   5.68165064e-01,
         3.46453667e-01,   2.52554804e-01,   6.12089336e-01,
        -4.72555965e-01,   6.74219608e-01,   1.13693979e-02,
         7.28994831e-02,

In [49]:
stemmer = PorterStemmer()
def w2v_vector(text):
        digits = re.compile(r"\d[\d\$\%\.]*")
        data_no_digits = digits.sub("<NUM>", text)
        # Take out not allowed characters
        not_allowed = re.compile(r"[^\s\w<>_]")
        data_allowed = not_allowed.sub("", data_no_digits)
        # Split to wirds
        words = data_no_digits.split()
        stemmed_words = [stemmer.stem(w) for w in words if w not in stopwords.words('english')]
        doc_vec = None
        for word in stemmed_words:
            try:
                if doc_vec is not None:
                    doc_vec += w2v.wv[word]
                    n+=1
                else:
                    doc_vec=np.copy(w2v.wv[word])
                    n=1
            except KeyError:
                continue
        return doc_vec/n

In [66]:
# df_w2v = asDataFrame(vectorizer=w2v_vector)

In [67]:
# df_w2v.head()

In [52]:
import joblib

In [68]:
# joblib.dump(df_w2v, filename='df_w2v')

In [53]:
df_w2v=joblib.load('df_w2v')

In [54]:
df_w2v.iloc[:5,:]

,vector,tag
num,,
39500,"[-1.61933, -0.464451, 0.0550538, -0.297373, -0...",bylaws
39510,"[-0.719852, 0.686502, -0.322379, 0.361197, -1....",bylaws
39520,"[-1.11765, -0.301801, 0.132282, -0.292014, -0....",bylaws
39530,"[-1.19122, -0.162094, -1.15168, -0.866537, -0....",bylaws
39540,"[-1.94795, -0.561751, 0.171561, -0.176203, -1....",bylaws


In [55]:
# Explode the vector to fetures
X = df_w2v.vector.apply(pd.Series)

In [56]:
X[:5]

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
num,,,,,,,,,,,,,,,,,,,,,
39500,-1.619327,-0.464451,0.055054,-0.297373,-0.805999,0.871753,-0.741109,0.678718,0.310186,0.592027,...,-1.676588,0.568165,0.346454,0.252555,0.612089,-0.472556,0.674220,0.011369,0.072899,0.807618
39510,-0.719852,0.686502,-0.322379,0.361197,-1.316045,-0.194554,-0.008509,0.825997,0.258301,0.356196,...,-1.385357,0.589663,0.132903,-0.332802,0.847092,-1.039762,1.117214,0.266728,0.623740,0.992927
39520,-1.117647,-0.301801,0.132282,-0.292014,-0.539731,1.928234,-0.809309,-0.283378,0.292591,0.357648,...,-1.932810,1.005382,-0.130626,-0.327796,1.007238,-0.960057,0.570307,0.149203,0.714382,0.933405
39530,-1.191220,-0.162094,-1.151682,-0.866537,-0.678396,-0.559103,0.389821,1.355353,0.845783,-0.100474,...,-1.232048,0.304298,0.443099,0.121708,0.459607,-0.805232,0.639606,0.563926,1.058884,0.451054
39540,-1.947951,-0.561751,0.171561,-0.176203,-1.474341,1.446521,-0.795225,0.738920,0.686716,0.489750,...,-1.820323,1.100861,0.151960,-0.037392,0.652698,-0.786405,0.680236,0.159842,0.143596,0.666396


In [57]:
y=df_w2v.tag

In [58]:
# Now modeling
lr_clf = LogisticRegression(C=30)

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=0)

In [60]:
lr_clf.fit(X_train,y_train)

LogisticRegression(C=30, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [61]:
pred = lr_clf.predict(X_test)

In [62]:
acc = accuracy_score(y_test, pred)
acc

0.99006323396567297

In [63]:
cm = confusion_matrix(y_test, pred)
cm

array([[ 47,   0,   0,   0,   0,   3],
       [  0, 109,   0,   0,   0,   0],
       [  0,   0,  50,   1,   0,   0],
       [  0,   0,   1, 341,   0,   2],
       [  0,   1,   0,   0,  32,   1],
       [  1,   0,   0,   1,   0, 517]], dtype=int64)

In [64]:
cv = cross_val_score(lr_clf, X, y, cv=10, verbose=1)

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    5.0s finished


In [65]:
np.mean(cv)

0.98870696591296225

Need to be completed with the test data - this is done in the submission file.